In [3]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [5]:
import os
import numpy as np
import h5py
import gc

from keras.preprocessing.image import ImageDataGenerator

from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16

from keras.models import Sequential,Model
from keras import applications
from keras.layers import Input
from keras import backend as K

In [6]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
predict_data_dir = 'data/test'

path_features = 'features/'
path_folder='Combined_prediction/'


batch_size = 64
epochs = 50

img_width, img_height = 224, 224

In [10]:
datagen = ImageDataGenerator(rescale=1. / 255)
features_generator_train = datagen.flow_from_directory(train_data_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode=None,
                                                shuffle=False)
features_generator_validation = datagen.flow_from_directory(validation_data_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode=None,
                                                shuffle=False)

features_generator_test = datagen.flow_from_directory(predict_data_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode=None,
                                                shuffle=False)

Found 27827 images belonging to 7 classes.
Found 11233 images belonging to 7 classes.
Found 1512 images belonging to 1 classes.


## Feature Extraction

In [11]:
def ExtractFeatures(MODEL,features_generator_train,features_generator_validation,features_generator_test):
    print('Extract features from', MODEL.__name__ )
    #channel-last is for tensorflow backend.
    input_tensor = Input(shape=(224, 224, 3))
    feature_model = MODEL(input_tensor=input_tensor, weights='imagenet', include_top=False)
    with h5py.File(path_features+"Features_%s.h5"%MODEL.__name__) as h:
        features_train = feature_model.predict_generator(features_generator_train)
        h.create_dataset("train", data=features_train)
        del features_train
        gc.collect()

        features_validation = feature_model.predict_generator(features_generator_validation)
        h.create_dataset("validation", data=features_validation)
        del features_validation
        gc.collect()

        features_test = feature_model.predict_generator(features_generator_test)
        h.create_dataset("test", data=features_test)
        del features_test
        gc.collect()

        h.create_dataset("label_train", data=features_generator_train.classes)
        h.create_dataset("label_validation", data=features_generator_validation.classes)
    print(MODEL.__name__,'feature saved')

In [12]:
for MODEL in [ResNet50,Xception,InceptionV3,VGG19,VGG16]:
    ExtractFeatures(MODEL,features_generator_train,features_generator_validation,features_generator_test)

Extract features from ResNet50
ResNet50 feature saved
Extract features from Xception
Xception feature saved
Extract features from InceptionV3
InceptionV3 feature saved
Extract features from VGG19
VGG19 feature saved
Extract features from VGG16
VGG16 feature saved


___

In [13]:
features = os.listdir(path_features )

for i,feature in enumerate(features):
    if i != 3:
        print(i,feature)
        with h5py.File(path_features+feature, 'r') as h:
            y_train = np.array(h['label_train'])
            y_validation = np.array(h['label_validation'])

            X_train = np.empty([len(y_train),0])
            X_validation = np.empty([len(y_validation),0])


            train=np.array(h['train'])
            shape=train.shape
            train=train.reshape(shape[0],shape[1]*shape[2]*shape[3])

            validation=np.array(h['validation'])
            shape=validation.shape
            validation=validation.reshape(shape[0],shape[1]*shape[2]*shape[3])

        X_train = np.concatenate((X_train,train),axis=1)
        del train
        gc.collect()

        X_validation=np.concatenate((X_validation,validation),axis=1)
        del validation
        gc.collect()

0 Features_Xception.h5


MemoryError: 

In [ ]:
print("X_train.shape",X_train.shape,
        "X_validation.shape",X_validation.shape,
        "y_train.shape",y_train.shape,
        "y_validation.shape",y_validation.shape)

In [ ]:
#training the top model


model = Sequential()
model.add(Dense(256, activation='relu',input_shape=X_train.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', metrics=['accuracy'])

hist_combined=model.fit(X_train, y_train,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(X_validation, y_validation)
          )



#same the weights
model.summary()
model.save_weights(path_folder+'3_combined_top_weight.h5')
print('Weight saved')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,10))
epochs = range(1, len(hist_combined.history['acc']) + 1)
plt.plot(epochs, hist_combined.history['loss'], label='Training Loss')
plt.plot(epochs, hist_combined.history['val_loss'], label='Validation Loss')

plt.title('Loss of Top combined 4 Features')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(path_folder+'3_loss_combined_4.jpg')
plt.show()
print('Plot saved')

plt.figure(figsize=(16,10))
epochs = range(1, len(hist_combined.history['acc']) + 1)
plt.plot(epochs, hist_combined.history['acc'], label='Training Accuracy')
plt.plot(epochs, hist_combined.history['val_acc'], label='Validation Accuracy')
plt.title('Accuracy of Top combined 4 Features')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(path_folder+'3_acc_combined_4.jpg')
plt.show()
print('Plot saved')

In [ ]:
# # combine the feature of test dataset and flatten
# for i,feature in enumerate(features):
#     if i != 3:
#         print(i,feature)
#         with h5py.File(path_features+feature, 'r') as h:
#             test=np.array(h['test'])
#             X_test = np.empty([len(test),0])
#             shape=test.shape
#             test=test.reshape(shape[0],shape[1]*shape[2]*shape[3])
#         X_test=np.concatenate((X_test,test),axis=1)
#         del test
#         gc.collect()
# print(X_test.shape)

# #predict the class
# pred_class = model.predict(X_test)
# print(pred_class[:5])

# #save the prediction
# ID_n = np.arange(1,len(X_test)+1)
# ID={'id':ID_n}


# test_img = os.listdir(predict_data_dir+os.sep+'test')
# print(test_img[0:5])

# prediction_label_df=pd.DataFrame(ID)
# prediction_label_df['label']=pred_class
# prediction_label_df['Image']=test_img

# print('Prediction save to ',path_folder)